# Домашнее задание 3. Детекция объектов

## Описание

Сыграем в квиддич? Или лучше в карты?

В этом дз вам предстоит написать практически с нуля архитектуру для детекции, а также воспользоваться готовым решением. На выбор даётся два датасета, отличаются они только картинками. Форматы, баллы - все одинаково.

Первый вариант это датасет по кадрам игры в квиддич из Гарри Поттера. Если вы забыли правила, то нажмите [сюда](https://harrypotter.fandom.com/ru/wiki/%D0%9A%D0%B2%D0%B8%D0%B4%D0%B4%D0%B8%D1%87). Вы научитесь искать и выделять на фотографиях бладжеры, квоффл и снитч.

Второй вариант это датасет с игральными картами. Если вы забыли что такое карты, то нажмите [сюда](https://ru.wikipedia.org/wiki/%D0%98%D0%B3%D1%80%D0%B0%D0%BB%D1%8C%D0%BD%D1%8B%D0%B5_%D0%BA%D0%B0%D1%80%D1%82%D1%8B). Вы научитесь искать и выделять на фотографиях несколько типов карт.

Оба варианта содержат около 300 картинок, данные хранятся в xml в формате PascalVOC. Есть малые отличия, но ничего страшного.


Если с самописным детектором совсем не получается, то можно после создания датасетов перейти к концу, где обучается готовый, с ним будет проще :)

### Notes

Дз проверялось на работоспособность в colab. Не гарантируется, что будет работать на чем-то другом, и точно не будет работать из коробки на Windows.

По вопросам формулировок (не ошибок торча!), в случае отсутствия ответа в общем чате (поиск по чату позволяет проверить), можно написать в него с тегом @markblumenau.

### Данные

Скачайте один из датасетов на свой вкус и начните работу с ним.
Разметка находится в xmls папке, картинки в images.

In [ ]:
# Cards
! rm -rf cards/
! rm -rf cards.zip

! wget https://github.com/markblumenau/hw3_iad_dl/raw/main/cards/data.zip -O cards.zip
! unzip -q cards.zip -d cards

! mv cards/data/* cards/
! rmdir cards/data

DATASET_NAME = 'cards'
N_EPOCHS = 15

### Зависимости

In [ ]:
import os
import glob
import json
import shutil
import random
import typing
import pathlib
import functools

import xml
import PIL
import tqdm
import numpy
import torch
import wandb
import matplotlib
import torchvision
import albumentations
import matplotlib.pyplot as plt
import albumentations.pytorch.transforms as transforms

device = torch.device(
    "cuda" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available()
    else "cpu"
)
print(device)

RANDOM_STATE = 42
def set_random_seed(seed: int) -> None:
    random.seed(seed)
    numpy.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.backends.cudnn.deterministic = True
def fix_random() -> None:
    return set_random_seed(RANDOM_STATE)
fix_random()

In [ ]:
wandb.login(anonymous = "allow")

## Задача 1. 0.5 балла.

Ниже написан код для стандартного Dataset из библиотеки pytorch. Dataset требует реализации `__getitem__` и `__len__` методов. Далее эти методы будут использованы для формирования батчей для обучения. Поскольку читать придется из xml файлов, нужно перед этим дописать функцию get_xml_data, чтобы по названию картинки подтягивать аннотации.

In [ ]:
class PascalDataset(torch.utils.data.Dataset):
    def __init__(self, root: pathlib.Path, train: bool, transform: albumentations.Compose):
        self.root = root
        self.train = train
        self.transform = transform
        assert self.root.is_dir(), f"No data at `{root}`"

        filepaths = numpy.array(glob.glob(root.joinpath("images/*").as_posix()))
        with open(root.joinpath("class_dict"), "r") as f: self.class_to_idx = json.load(f)
        self.idx_to_class = { v: k for k, v in self.class_to_idx.items() }

        fix_random()
        split = int(len(filepaths) * 0.9)
        permutation = numpy.random.permutation(len(filepaths))
        indexes = permutation[:split] if train else permutation[split:]
        self.images = [ pathlib.Path(filepath) for filepath in filepaths[indexes] ]

    def get_image(self, idx: int) -> numpy.ndarray[int]:
        return numpy.asarray(PIL.Image.open(self.images[idx]))
    
    def get_bboxes(self, idx: int) -> typing.List[typing.Tuple[int, int, int, int, int]]:
        xml_path = list(self.images[idx].parts)
        xml_path[1] = 'xmls'
        xml_path = pathlib.Path(*xml_path).with_suffix(".xml")

        bboxes = []
        for bbox in xml.etree.ElementTree.parse(xml_path).getroot().findall("object"):
            assert len(bbox.findall("name")) == 1
            assert len(bbox.findall("bndbox")) == 1
            bndbox = bbox.find("bndbox")
            class_name = bbox.find("name").text
            assert class_name in self.class_to_idx

            assert len(bndbox.findall("xmin")) == 1
            assert len(bndbox.findall("ymin")) == 1
            assert len(bndbox.findall("xmax")) == 1
            assert len(bndbox.findall("ymax")) == 1

            bboxes.append((
                int(bndbox.find("xmin").text), int(bndbox.find("ymin").text),
                int(bndbox.find("xmax").text), int(bndbox.find("ymax").text),
                self.class_to_idx[class_name]
            ))

        return bboxes
        
    def __getitem__(self, idx: int) -> typing.Tuple[torch.Tensor, typing.List[typing.Tuple[int, int, int, int, int]]]:
        data = self.transform(image = self.get_image(idx), bboxes = self.get_bboxes(idx))
        return data['image'], data['bboxes']

    def __len__(self) -> int:
        return len(self.images)

Ниже определяем стандартные нормализации и приведение размера к 512x512.


In [ ]:
mean = numpy.array([ 0.485, 0.456, 0.406 ])
std = numpy.array([ 0.229, 0.224, 0.225 ])

transforms = [
    albumentations.Resize(512, 512),
    albumentations.augmentations.transforms.Normalize(mean = mean, std = std),
    albumentations.pytorch.transforms.ToTensorV2(),
]

train_transform = albumentations.Compose(transforms, bbox_params = dict(format = "pascal_voc", min_visibility = 0.3))
test_transform = albumentations.Compose(transforms, bbox_params = dict(format = "pascal_voc", min_visibility = 0.5))

train_set = PascalDataset(pathlib.Path(DATASET_NAME), True, train_transform)
test_set = PascalDataset(pathlib.Path(DATASET_NAME), False, test_transform)

## Задача 2. 1 балл.

Теперь, когда мы загрузили данные, хорошо бы посмотреть на них, прежде чем обучать какие-либо модели. Напишите функцию `visualize`, которая принимает списки изображений и прямоугольников в качестве входных данных и рисует эти прямоугольники на изображениях.


Полезные функции:
* [plt.subplots](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.subplots.html) -- легко создавать несколько изображений в одной pyplot figure
* [ax.imshow](https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.imshow.html) -- отображение графиков (не забудьте откатить нормализацию)
* [ax.text](https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.text.html), [patches.Rectangle](https://matplotlib.org/stable/api/_as_gen/matplotlib.patches.Rectangle.html) -- для рисования прямоугольников и текста с аннотацией

In [ ]:
def get_class_color(class_name: str) -> str:
    match class_name:
        # Harry potter dataset
        case 'snitch': return 'gold'
        case 'quaffle': return 'red'
        case 'bludger': return 'blue'

        # Cards dataset
        case 'king': return 'red'
        case 'jack': return 'blue'
        case 'ace': return 'black'
        case 'ten': return 'green'
        case 'nine': return 'white'
        case 'queen': return 'pink'

        # Default
        case _: raise NotImplementedError
    
def visualize(data: typing.List[typing.Tuple[torch.Tensor, typing.List[typing.Tuple[int, int, int, int, int]]]]) -> None:
    denormalize = albumentations.Compose([
        albumentations.augmentations.transforms.Normalize(mean = [ 0., 0., 0. ], std = 1 / std, max_pixel_value = 1),
        albumentations.augmentations.transforms.Normalize(mean = -mean, std = [ 1, 1, 1 ], max_pixel_value = 1)
    ])

    fig, axes = matplotlib.pyplot.subplots(2, len(data) // 2 + len(data) % 2, figsize = (25, 11), dpi = 100)
    for i, ax in enumerate(axes.reshape(-1)):
        ax.axis(False)
        if i >= len(data): break
        image, bboxes = data[i]
        ax.imshow(denormalize(image = image.permute(1, 2, 0).numpy())['image'])
        for bbox in bboxes:
            xmin, ymin, xmax, ymax, class_idx = bbox
            class_name = train_set.idx_to_class[class_idx]
            class_color = get_class_color(class_name)
            rect = matplotlib.patches.Rectangle(
                (xmin, ymin), xmax - xmin, ymax - ymin,
                linewidth = 1, edgecolor = class_color, facecolor = 'none'
            )
            ax.add_patch(rect)
            ax.text(xmin, ymin - 5, class_name, color = class_color)

    fig.tight_layout()

visualize([ train_set[i] for i in range(10) ])

## Задача 3. 3 балла.

### YOLO-like детектор

Сейчас нам предстоить реализовать детектор, похожий на YOLO. Это один из самых простых детекторов с точки зрения реализации. YOLO описан в статье: [You Only Look Once: Unified, Real-Time Object Detection](https://arxiv.org/abs/1506.02640). Здесь мы его немного изменим и упростим. Будем использовать ResNet для извлечения признаков. На выходе мы будем получать карту признаков размера 16x16.

We convert lists of bounding boxes to the target downsampled grid. For this we

* compute centers of bounding boxes ($c_x, c_y$)
* change center coordinates (offset from the top left corner for each corresponding grid cell on a small grid)
* normalize box height and with to $[0, 1]$
* fill the target grid with values

### Задача 3.1. 1 балл.

Первым делом нам нужно реализовать collate function. Это функция позволит нам кастомизировать, как именно батч конструируется из примеров (смотрите [pytorch docs](https://pytorch.org/docs/stable/data.html#dataloader-collate-fn) для деталей).

Это функция должна принять на вход лист прямоугольников и сгенерировать тензор размера Bx16x16x6. Первая размерность - это количество примеров в батче. Далее идут две пространственные размерности, это сетка 16 на 16. 

В каналах у нас будут записаны:
* Сдвиги центра bbox относительно начала клеточки (клеточка это "пиксель" на изображении 16 на 16 на выходе сети). Записаны эти сдвиги будут в клеточку, к которой относятся. 2 канала (X, Y)
* Нормализованные ширина и высота bbox. 2 канала (W, H)
* Confidence сетки. Им мы будем пользоваться, чтобы фильтровать уверенность сетки в наличии bbox в данной клетке. Таргет содержит 1 там, где bbox есть, и 0 иначе. 1 канал
* Класс детекции (тот самый int, полученный из строки с названием)

In [ ]:
def collate_fn(
        batch: typing.List[typing.Tuple[torch.Tensor, typing.List[typing.Tuple[int, int, int, int, int]]]],
        downsample: int = 32
    ) -> typing.Tuple[torch.Tensor, torch.Tensor]:
    images, batch_boxes = zip(*batch)
    images = torch.stack(images)
    b, c, h, w = images.shape

    targets = images.new_zeros(b, 6, h // downsample, w // downsample)
    for i, boxes in enumerate(batch_boxes):
        xmin, ymin, xmax, ymax, classes = map(torch.squeeze, torch.split(images.new_tensor(boxes), 1, dim = -1))
        center_x = (xmin + xmax) / 2
        center_y = (ymin + ymax) / 2

        cell_x = (center_x // downsample).int()
        cell_y = (center_y // downsample).int()

        offset_x = (center_x - cell_x * downsample) / downsample
        offset_y = (center_y - cell_y * downsample) / downsample

        width = (xmax - xmin) / w
        height = (ymax - ymin) / h

        confidence = torch.ones_like(offset_x)
        targets[i, :, cell_x, cell_y] = torch.stack([ offset_x, offset_y, width, height, confidence, classes ])

    return images, targets

In [ ]:
def test_collate_fn() -> None:
    target1 = [ (100, 200, 200, 300, 2) ]
    target2 = [ (0, 250, 200, 300, 0), (0, 100, 100, 300, 1) ]
    imgs, targets = collate_fn([ (torch.rand((3, 512, 512)), target1), (torch.rand((3, 512, 512)), target2) ])
    assert imgs.shape == (2, 3, 512, 512)
    assert targets.shape == (2, 6, 16, 16)
    assert numpy.allclose(targets[0, :, 4, 7], torch.tensor([ 22 / 32, 26 / 32, 100 / 512, 100 / 512, 1, 2 ]))
    assert numpy.allclose(targets[1, :, 3, 8], torch.tensor([ 4 / 32, 19 / 32, 200 / 512, 50 / 512, 1, 0 ]))
    assert numpy.allclose(targets[1, :, 1, 6], torch.tensor([ 18 / 32, 8 / 32, 100 / 512, 200 / 512, 1, 1 ]))
    targets[0, :, 4, 7] = targets[1, :, 3, 8] = targets[1, :, 1, 6] = torch.zeros(6)
    assert numpy.allclose(targets, 0)
test_collate_fn()

Ниже вы можете увидеть пример, как выглядит решетка размера 16 на 16 на исходном изображении:

In [ ]:
def show(i: int) -> None:
    fig, ax = plt.subplots(figsize = (5, 5))

    img = train_set[i][0].permute(1, 2, 0) * torch.tensor(std).view(1, 1, -1) + torch.tensor(mean).view(1, 1, -1)
    bboxes = torch.tensor(train_set[i][1])

    ax.imshow(img)
    loc = plt.matplotlib.ticker.MultipleLocator(base = 32)
    ax.xaxis.set_major_locator(loc)
    ax.yaxis.set_major_locator(loc)
    ax.grid(which = "major", axis = "both", linestyle = "-", linewidth = 3)

    for bbox in bboxes:
        xmin, ymin, xmax, ymax = bbox[:-1]
        w = xmax - xmin
        h = ymax - ymin
        ax.add_patch(matplotlib.patches.Rectangle((xmin, ymin), w, h, fill = False, color = "red"))

    cx = (bboxes[:, 0] + bboxes[:, 2]) / 2
    cy = (bboxes[:, 1] + bboxes[:, 3]) / 2
    ax.scatter(cx, cy, color = "green", marker = "o")
show(5)

### Задача 3.2. 0.5 балла.

Выход нашей сетки будет несколько больше, чем Bx16x16x6. Почему? 

Мы решаем задачу, где классов больше одного. Вспомним прошлое дз: target был одним числом, но выход сетки содержал длинный-длинный вектор, из которого мы получали вероятность принадлежности к тому или иному классу. Здесь то же самое, но как бы в двумерии: у каждой клеточки из этих 16*16 будет свой вектор длины C, который мы будем использовать для определения класса.

Реализуйте обратное относительно collate_fn преобразования, чтобы декодировать выход нейронной сети. Применив функцию decode_prediction к выходу collate function вы должны получить изначальный набор прямоугольников с корректными размерами и координатами, а также классами. Применив к выходу нейросети мы тоже должны получить набор прямоугольников и тоже с корректными классами. 

То есть, нужно проделать операции из collate_fn в обратную сторону, но учесть, что у неройнки выход будет чуть длиннее, и там мы должны брать argmax для определения класса.

Hint: в target classes идут в конце. В нейронке они тоже будут в конце, но их будет больше 1. Можно проверять число каналов пришедшего объекта, если оно 6, то перед нами target и надо брать значение, которое записано в клеточке. Иначе (каналов больше 6) перед нами выход нейронки, и надо брать самый вероятный из них.

In [ ]:
def decode_predictions(
        pred: torch.Tensor,
        upsample: int = 32,
        threshold: float = 0.7
    ) -> typing.List[typing.Tuple[int, int, int, int, int]]:
    b, c, h, w = pred.shape
    batch_boxes = [ ]
    for i in range(b):
        indices = torch.nonzero(pred[i, 4, :, :] >= threshold)
        cell_x, cell_y = indices.transpose(0, 1)
        boxes = pred[i, :, cell_x, cell_y]

        center_x = (boxes[0, :] + cell_x) * upsample
        width = boxes[2, :] * w * upsample
        xmin = center_x - width / 2
        xmax = center_x + width / 2
        
        center_y = (boxes[1, :] + cell_y) * upsample
        height = boxes[3, :] * h * upsample
        ymin = center_y - height / 2
        ymax = center_y + height / 2
    
        classes = torch.argmax(boxes[5:, :], dim = 0) if boxes.shape[0] != 6 else boxes[5, :]
        boxes = torch.stack([ xmin, ymin, xmax, ymax, classes ]).transpose(0, 1)
        batch_boxes.append(list(map(tuple, boxes.tolist())))
    return batch_boxes

In [ ]:
def test_decode_predictions() -> None:
    target1 = [ (100, 200, 200, 300, 2) ]
    target2 = [ (0, 250, 200, 300, 0), (0, 100, 100, 300, 1) ]
    _, targets = collate_fn([ (torch.rand((3, 512, 512)), target1), (torch.rand((3, 512, 512)), target2) ])
    # Bboxes for target2 change order for some reason. However, it is clearly not important
    assert decode_predictions(targets) == [ target1, [ target2[1], target2[0] ] ]
test_decode_predictions()

### Задача 3.3. 1 балл.

Реализуйте модель. Первым делом примените первые 4 блока (до layer4 включительно) ResNet50. Далее добавьте несколько блоков (Conv2D, BatchNorm2D, ReLU). Постепенно уменьшайте количество каналов до 5+C, а размер изображения до 16 на 16. Например, 2048 -> 512 -> 128 -> 32 -> 5+C, где С - количество классов в вашем датасете. Размер ядра при этом 3, паддинг 1. Но вариантов много, попробуйте разные! **Последним слоем обязательно должна быть свертка.** Так как все значения, которые мы предсказываем, находятся в отрезке от 0 до 1 (благодаря нормировке с клеточками), мы после финальной свертки еще применим сигмоиду. Для классов в такой постановке это не навредит.

Если будете фантазировать, то для получения правильного размера изображения после сети не стесняйтесь применять слои с фильтрами больше 3.

In [ ]:
list(map(lambda item: item[0], torchvision.models.resnet50().named_children()))

In [ ]:
def make_block(in_channels: int, out_channels: int) -> torch.nn.Module:
    return torch.nn.Sequential(
        torch.nn.Conv2d(in_channels, out_channels, kernel_size = 3, padding = 1),
        torch.nn.BatchNorm2d(out_channels),
        torch.nn.ReLU()
    )

def Detector(num_classes: int) -> torch.nn.Module:
    resnet = torchvision.models.resnet50(weights = torchvision.models.ResNet50_Weights.DEFAULT)
    return torch.nn.Sequential(
        *(list(resnet.children())[:-2]),
        make_block(2048, 512),
        make_block(512, 128),
        make_block(128, 32),
        torch.nn.Conv2d(32, 5 + num_classes, kernel_size = 3, padding = 1),
        torch.nn.Sigmoid()
    )

In [ ]:
def test_detector() -> None:
    assert Detector(3)(torch.rand((5, 3, 512, 512))).shape == torch.Size([ 5, 8, 16, 16 ])
test_detector()

### Задача 3.4. 0.5 балла.



Реализуйте функцию потерь.

Для этого:
* Сделайте маску, которая будет говорить о положении детектируемых объектов. Её нужно использовать с помощью masked_select (см. доки PyTorch)
* Лосс похож на оригинальный для Yolo V1 и состоит из 4 частей (reduction='sum' для всех)
    - localization loss - Мы берем MSE по координатам бокса там, где есть детектируемый объект
    - box_loss - MSE от корней ширины и высоты bbox там, где есть детектируемый объект
    - classification_loss - Если детектируемый объект есть, то его кросс-энтропия по его классу
    - confidence_loss - Бинарная кросс-энтропия факта наличия объекта ДЛЯ ВСЕХ пикселей. Делается отдельно для детектируемых объектов (вес 1) и для недетектируемых (вес 0.1 например, поскольку их гораздо больше, но можно экспериментировать)

* Ниже есть assert. Если вы экспериментируете с лоссом, он не будет проходить, не обращайте на него внимание. Если будете делать описанное выше, то учтите reduction. Бинарная кросс-энтропия вызывается через BCELoss. Параметр C используется для задачи числа классов. assert написан для 3 классов, в задаче с картами их 6. Подумайте как зависит индексация от параметра C и используйте его.

In [ ]:
def special_loss(
        preds: torch.Tensor,
        targets: torch.Tensor,
        check: bool = False
    ) -> typing.Union[torch.Tensor, typing.Tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]]:
    assert preds.shape[0] == targets.shape[0]
    assert preds.shape[2] == targets.shape[2]
    assert preds.shape[3] == targets.shape[3]
    assert targets.shape[1] == 6

    indices = torch.nonzero(targets[:, 4, :, :] == 1)
    b, x, y = indices.transpose(0, 1)
    detected_preds = preds[b, :, x, y]
    detected_targets = targets[b, :, x, y]

    indices = torch.nonzero(targets[:, 4, :, :] == 0)
    b, x, y = indices.transpose(0, 1)
    undetected_preds = preds[b, :, x, y]
    undetected_targets = targets[b, :, x, y]
    
    assert detected_targets.shape[0] + undetected_targets.shape[0] == preds.shape[0] * preds.shape[2] * preds.shape[3]

    localization_loss = torch.nn.functional.mse_loss(detected_preds[:, 0:2], detected_targets[:, 0:2], reduction = 'sum')
    box_loss = torch.nn.functional.mse_loss(torch.sqrt(detected_preds[:, 2:4]), torch.sqrt(detected_targets[:, 2:4]), reduction = 'sum')
    classification_loss = torch.nn.functional.cross_entropy(detected_preds[:, 5:], detected_targets[:, 5].long(), reduction = 'sum')
    confidence_loss_1 = torch.nn.functional.binary_cross_entropy(detected_preds[:, 4], detected_targets[:, 4], reduction= 'sum')
    confidence_loss_2 = torch.nn.functional.binary_cross_entropy(undetected_preds[:, 4], undetected_targets[:, 4], reduction = 'sum')
    confidence_loss = confidence_loss_1 + 0.1 * confidence_loss_2

    if check: return localization_loss, box_loss, classification_loss, confidence_loss
    else: return localization_loss + box_loss + classification_loss + confidence_loss

In [ ]:
# localization box classification confidence - возвращаются в таком порядке, можно сравнить
assert special_loss(torch.zeros((10, 8, 16, 16)), torch.ones((10, 6, 16, 16)), check = True) == (torch.tensor(5120.), torch.tensor(5120.), torch.tensor(2812.4465), torch.tensor(256000.))

## Задача 4. 2 балла.

### Обучим модель

Обучите вашу модель (написав цикл обучения), и покажите что она работает (скорее всего, объекты найдутся на 1-2 картинках).

In [ ]:
def train(model: torch.nn.Module, n_epochs: int, device: torch.device) -> None:
    model = model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr = 1e-3)
    train_loader = torch.utils.data.DataLoader(train_set, batch_size = 10, collate_fn = collate_fn, shuffle = True)

    plot = [ ]
    for epoch in range(n_epochs):
        sum_loss = 0
        for (images, targets) in tqdm.tqdm(train_loader, desc = 'Epoch {}'.format(epoch + 1)):
            model.train() # Enter train mode
            optimizer.zero_grad() # Zero gradients
            output = model(images.to(device)) # Get predictions
            loss = special_loss(output, targets.to(device))
            loss.backward() # Calculate gradients
            optimizer.step() # Update weights
            sum_loss += loss.item()
            plot.append(loss.item())
        avg_loss = sum_loss / len(train_loader)
        print(f"Epoch {epoch + 1} done; Train loss {avg_loss:.3f};")
    matplotlib.pyplot.plot(plot)

In [ ]:
fix_random()
model = Detector(len(train_set.class_to_idx))
train(model, 20, device)

### Посмотрим результаты

Запустим обученный детектор на тестовых изображениях:

In [ ]:
test_loader = torch.utils.data.DataLoader(test_set, batch_size = 10, collate_fn = collate_fn)
images, targets = next(iter(test_loader))
targets = decode_predictions(targets)
visualize(list(zip(images, targets)))

In [ ]:
model.eval()
output = model(images.to(device)).cpu().detach()
predictions = decode_predictions(output, threshold = 0.05)
visualize(list(zip(images, predictions)))

Результат сильно так себе, да? Есть множество вариантов улучшений, самый простой из которых это приделать к выходу [NMS](https://paperswithcode.com/method/non-maximum-suppression#:~:text=Non%20Maximum%20Suppression%20is%20a,below%20a%20given%20probability%20bound.). Если хочется, можно почитать про YOLO v1 [тут](https://arxiv.org/abs/1506.02640).

## Задача 5. 3.5 балла.

Займёмся более простыми вещами. Возьмем готовую архитектуру, обучим её на наших данных и посмотрим.

Для этого будем использовать YOLO v8 от ultralytics.

In [ ]:
! pip install ultralytics

### Задача 5.1. 1.5 балла.

Чтобы дальше модель обучалась одной строкой, данные нужно переложить в правильный формат. Да-да, классика перекладывания JSON. Как правильно паковать можно посмотреть [тут](https://roboflow.com/formats/yolov8-pytorch-txt).

Если коротко:
* Есть .yaml, где живут пути к папкам с картинками, количество классов и их названия
* Есть папочки train valid (их поможем вам собрать), в них две подпапки:
    - Первая images, в ней лежат картинки
    - Вторая labels, в ней лежат файлы с названиями как у картинок, но вместо расширения картинок нужен .txt, внутри формат как описан на Roboflow

In [ ]:
os.makedirs("{}/train/images".format(DATASET_NAME), exist_ok = True)
os.makedirs("{}/train/labels".format(DATASET_NAME), exist_ok = True)
os.makedirs("{}/valid/images".format(DATASET_NAME), exist_ok = True)
os.makedirs("{}/valid/labels".format(DATASET_NAME), exist_ok = True)

Реализуйте функцию, которая принимает аннотации в изначальном формате, а возвращает их в нужном для YOLO v8. Это должен быть массив готовых строк, которые можно сразу забрасывать в файлик, добавив \n.

In [ ]:
def annotation2txt(image_width: int, image_height: int, bbox: typing.Tuple[int, int, int, int, int]) -> str:
    xmin, ymin, xmax, ymax, class_idx = bbox

    center_x = (xmin + xmax) / (2 * image_width)
    center_y = (ymin + ymax) / (2 * image_height)

    width = (xmax - xmin) / image_width
    height = (ymax - ymin) / image_height

    return '{} {} {} {} {}'.format(class_idx, center_x, center_y, width, height)


def annotations2txt(bboxes: typing.List[typing.Tuple[int, int, int, int, int]], width: int, height: int) -> typing.Iterable[str]:
    return map(functools.partial(annotation2txt, width, height), bboxes)


def process(dataset: PascalDataset, name: str) -> None:
    for i in tqdm.trange(len(dataset), desc = name):
        path = dataset.images[i]
        image = dataset.get_image(i)
        bboxes = dataset.get_bboxes(i)
        shutil.copyfile(path, "./{}/{}/images/{}".format(DATASET_NAME, name, path.name))
        with open("./{}/{}/labels/{}.txt".format(DATASET_NAME, name, path.stem), "w", encoding = "utf8") as f:
            f.write("\n".join(annotations2txt(bboxes, image.shape[1], image.shape[0])))

process(train_set, "train")
process(test_set, "valid")

In [ ]:
def build_yaml(classes: typing.List[str]):
    path = "path: ../{}".format(DATASET_NAME)
    train = "train: ./train/images"
    val = "val: ./valid/images"
    nc = "nc: {}".format(len(classes))
    names = "names: {}".format(classes)
    with open("{}/data.yaml".format(DATASET_NAME), "w") as f:
        f.write(f"{path}\n{train}\n{val}\n\n{nc}\{names}")
build_yaml(list(train_set.class_to_idx.keys()))

### Задание 5.2. 1.5 балла.

Обучите модель YOLO v8 самого маленького размера. Библиотека максимально friendly, от вас требуется написать две строчки. Модель нужно взять необученную!

Подсказка: подумайте зачем вам data.yaml и что такое yolov8n.yaml (не стесняйтесь гуглить)

In [ ]:
import ultralytics
model = ultralytics.YOLO('yolov8n.yaml')
model.train(
    pretrained = False, data = './{}/data.yaml'.format(DATASET_NAME),
    seed = RANDOM_STATE, deterministic = True, workers = 1,
    project = "DL-HW-3", name = DATASET_NAME, exist_ok = True
)

### Задание 5.3. 0.5 балла.

Как-нибудь отрисуйте предсказания на валидационной выборке (хотя бы части из 5-10 картинок).

Здесь можно использовать костыли с параметром save=True у predict, потом прочитать их чем-нибудь, отрисовать матплотлибом. Есть варианты и получше. Дефолтный show будет пытаться показывать через opencv imshow, он в коллабе работать не будет.

In [ ]:
def visualize_ultralytics(test_set: PascalDataset, rows: int, cols: int):
    fig, axes = matplotlib.pyplot.subplots(rows, cols, figsize = (25, 11), dpi = 100)
    for i, ax in enumerate(axes.reshape(-1)):
        ax.axis(False)
        # https://docs.ultralytics.com/modes/predict/#plotting-results
        prediction = model.predict(test_set.images[i])[0].plot()
        axes.reshape(-1)[i].imshow(PIL.Image.fromarray(prediction[..., ::-1]))
    fig.tight_layout()
visualize_ultralytics(test_set, 2, 5)